# Pipeline Project Rafael Eguren

### En este proyecto voy a hacer un análisis del mercado de coches de segunda mano en España. Para este análisis voy a usar el siguente dataset cogido de Kaggle: https://www.kaggle.com/harturo123/online-adds-of-used-cars Según la información de Kaggle, este dataset ha sido extraído de la web especializada en coches https://www.autoscout24.com/ a fecha 18/03/2018.
### Voy a comenzar limpiando el dataset, posteriormente lo completaré con una api y finalmente sacaré una tabla con las conclusiones. En otro archivo llamado Pipeline_Rafael_Eguren.py podremos sacar las conclusiones a través de terminal gracias al pipeline desarrollado en ese archivo.
### El objetivo de este análisis es conseguir una tabla que permita ver rápidamente y de forma sencilla como está configurado el parque de coches de segunda mano en España, sacando las marcas y modelos que los componen, cuantas unidades hay de cada modelo y obteniendo una media de los kilómetros, potencia, antigüedad y precio de cada modelo de coche de segunda mano.

Importo las librerías que voy a usar en esta primera parte.

In [19]:
import pandas as pd
import numpy as np

Leo el archivo en fomato .csv que voy a utilizar.

In [21]:
cars = pd.read_csv('./data.csv', sep=';')
cars.head()

,ID,make,model,version,months_old,power,sale_type,num_owners,gear_type,fuel_type,kms,price
0,97860,Porsche,911,Carrera 4 S Coup�,240.0,210.0,classic,3.0,manual,gasoline,202000.0,999999
1,27821,Ford,Mustang,Gt500 Cabrio Vendido,54.0,487.0,used,1.0,manual,gasoline,30000.0,685000
2,97801,Porsche,911,3.3 Turbo Targa (Precio A Consultar),358.0,220.0,used,NaN,manual,gasoline,56300.0,555555
3,98251,Porsche,911,R Unidad 343 De 991-Iva Deducible,14.0,368.0,used,1.0,manual,gasoline,2800.0,470000
4,98250,Porsche,911,Gt2 Rs,3.0,515.0,used,NaN,NaN,gasoline,10.0,450000


Miro si hay duplicados y los elimino en caso de que los haya.

In [22]:
before = len(cars)
cars1 = cars.drop_duplicates()
after = len(cars1)
print('Number of duplicate records dropped: ', str(before - after))

Number of duplicate records dropped:  0


Selecciono las columnas que me quiero quedar para mi análisis.

In [23]:
cars1 = cars1[['make', 'model', 'months_old', 'power','gear_type','fuel_type','kms','price']]
cars1.head()

,make,model,months_old,power,gear_type,fuel_type,kms,price
0,Porsche,911,240.0,210.0,manual,gasoline,202000.0,999999
1,Ford,Mustang,54.0,487.0,manual,gasoline,30000.0,685000
2,Porsche,911,358.0,220.0,manual,gasoline,56300.0,555555
3,Porsche,911,14.0,368.0,manual,gasoline,2800.0,470000
4,Porsche,911,3.0,515.0,NaN,gasoline,10.0,450000


Veo que la columna que me indica la antigüedad del vehículo está en meses, creo que en estos casos es mejor tener la antigüedad en años, por lo que creo una nueva columna que me indique su atigüedad en años. Para ellos cojo la columna que está en meses y la divido entre 12 para sacar los años.

In [24]:
cars1['old_years'] = cars1['months_old'].divide(12).round()
cars1.head()

,make,model,months_old,power,gear_type,fuel_type,kms,price,old_years
0,Porsche,911,240.0,210.0,manual,gasoline,202000.0,999999,20.0
1,Ford,Mustang,54.0,487.0,manual,gasoline,30000.0,685000,4.0
2,Porsche,911,358.0,220.0,manual,gasoline,56300.0,555555,30.0
3,Porsche,911,14.0,368.0,manual,gasoline,2800.0,470000,1.0
4,Porsche,911,3.0,515.0,NaN,gasoline,10.0,450000,0.0


Una vez creada la nueva columna en años, elimino la columna de los meses, ya que no me hace falta.

In [25]:
cars1 = cars1[['make', 'model', 'power','gear_type','fuel_type','kms','old_years','price']]
cars1.head()

,make,model,power,gear_type,fuel_type,kms,old_years,price
0,Porsche,911,210.0,manual,gasoline,202000.0,20.0,999999
1,Ford,Mustang,487.0,manual,gasoline,30000.0,4.0,685000
2,Porsche,911,220.0,manual,gasoline,56300.0,30.0,555555
3,Porsche,911,368.0,manual,gasoline,2800.0,1.0,470000
4,Porsche,911,515.0,NaN,gasoline,10.0,0.0,450000


Voy a mirar si todos los valores únicos de la columna 'make' son correctos.

In [26]:
print(set(cars1['make']))

{'Volvo', 'Jaguar', 'Renault', 'Jeep', 'Peugeot', 'Dacia', 'Porsche', 'Honda', 'Chevrolet', 'Bmw', 'Alfa', 'Hyundai', 'Audi', 'Nissan', 'Mitsubishi', 'Land', 'Fiat', 'Ds', 'Ford', 'Ssangyong', 'Mini', 'Chrysler', 'Bentley', 'Volkswagen', 'Toyota', 'Suzuki', 'Saab', 'Smart', 'Mercedes-Benz', 'Skoda', 'Kia', 'Opel', 'Seat', 'Iveco', 'Citroen', 'Mazda'}


Hago lo mismo pero con los valores de la columna 'model'.

In [27]:
print(set(cars1['model']))

{'Rav4', '500', 'V220', 'E320', 'Berlingo', 'Polo', 'Sharan', 'Talisman', 'Spacetourer', 'I10', 'A7', 'Asx', 'Meriva', 'Corsa', 'XsaraPicasso', 'Xc90', 'Tt', '640', 'Ml350', 'Forfour', 'C2', 'Punto', 'Touareg', '6', 'Continental', 'RomeoMito', 'Verso', 'A1', 'A3', 'X5', 'Leon', 'C5', '216', 'GrandScenic', 'Panamera', '5008', 'X-Trail', 'Qashqai', 'Ix35', 'RoverRangeRover', 'Kuga', 'Beetle', 'Octavia', 'Mondeo', 'Civic', 'AutomobilesDs5', '3008', 'Rexton', '120', 'Pulsar', 'Boxster', 'Expert', 'Gla200', 'LandCruiser', '911', 'Kangoo', 'RoverFreelander', '730', 'RoverDiscovery', 'C-Max', '325', 'S40', 'Cordoba', 'Sandero', 'E350', 'I20', 'E220', 'Golf', 'Focus', 'X3', 'PassatVariant', 'Q3', 'Carnival', 'Ka/Ka+', 'Aveo', 'Transporter', '530', 'Mokka', 'S60', 'Compass', 'S-Max', 'GrandC4Picasso', 'Ateca', 'Vectra', '500C', 'C4Picasso', 'Jumpy', 'I30', 'Rio', 'A4Allroad', 'Zafira', 'C4', 'TransitConnect', 'I40', 'V60', 'Q2', 'Q5', 'B-Max', 'Touran', '116', 'Wrangler', 'Scenic', 'V40Cc', 'Ro

Veo que hay un valor que me aporta un símbolo extraño, como seguramente me vaya a dar problemas, decido reemplazar ese nombre por otro con la función .replace.

In [28]:
cars1['model'] = cars1['model'].str.replace('C-Elys�e', 'C-Elys')
print(set(cars1['model']))

{'Rav4', '500', 'V220', 'E320', 'Berlingo', 'Polo', 'Sharan', 'Talisman', 'Spacetourer', 'I10', 'A7', 'Asx', 'Meriva', 'Corsa', 'XsaraPicasso', 'Xc90', 'Tt', '640', 'Ml350', 'Forfour', 'C2', 'Punto', 'Touareg', '6', 'Continental', 'RomeoMito', 'Verso', 'A1', 'A3', 'X5', 'Leon', 'C5', '216', 'GrandScenic', 'Panamera', '5008', 'X-Trail', 'Qashqai', 'Ix35', 'RoverRangeRover', 'Kuga', 'Beetle', 'Octavia', 'Mondeo', 'Civic', 'AutomobilesDs5', '3008', 'Rexton', '120', 'Pulsar', 'Boxster', 'Expert', 'Gla200', 'LandCruiser', '911', 'Kangoo', 'RoverFreelander', '730', 'RoverDiscovery', 'C-Max', '325', 'S40', 'Cordoba', 'Sandero', 'E350', 'I20', 'E220', 'Golf', 'Focus', 'X3', 'PassatVariant', 'Q3', 'Carnival', 'Ka/Ka+', 'Aveo', 'Transporter', '530', 'Mokka', 'S60', 'Compass', 'S-Max', 'GrandC4Picasso', 'Ateca', 'Vectra', '500C', 'C4Picasso', 'Jumpy', 'I30', 'Rio', 'A4Allroad', 'Zafira', 'C4', 'TransitConnect', 'I40', 'V60', 'Q2', 'Q5', 'B-Max', 'Touran', '116', 'Wrangler', 'Scenic', 'V40Cc', 'Ro

Este es el resultado del dataset hasta el momento con los cambios realizados.

In [29]:
cars1.head()

,make,model,power,gear_type,fuel_type,kms,old_years,price
0,Porsche,911,210.0,manual,gasoline,202000.0,20.0,999999
1,Ford,Mustang,487.0,manual,gasoline,30000.0,4.0,685000
2,Porsche,911,220.0,manual,gasoline,56300.0,30.0,555555
3,Porsche,911,368.0,manual,gasoline,2800.0,1.0,470000
4,Porsche,911,515.0,NaN,gasoline,10.0,0.0,450000


### Agregación de una Api

Importo las librerías que voy a usar.

In [30]:
import json
import requests

En este punto me encuentro con el mayor problema que he tenido en este proyecto. Me ha resultado difícil encontrar una Api que se ajustara a mi dataset y que le pudiera añadir información. Después de buscar y estar dando muchas vueltas, decido utilizar la siguiente Api: 

In [31]:
response = requests.get('https://parallelum.com.br/fipe/api/v1/carros/marcas')
results = response.json()
results

[{'nome': 'Acura', 'codigo': '1'},
 {'nome': 'Agrale', 'codigo': '2'},
 {'nome': 'Alfa Romeo', 'codigo': '3'},
 {'nome': 'AM Gen', 'codigo': '4'},
 {'nome': 'Asia Motors', 'codigo': '5'},
 {'nome': 'ASTON MARTIN', 'codigo': '189'},
 {'nome': 'Audi', 'codigo': '6'},
 {'nome': 'Baby', 'codigo': '207'},
 {'nome': 'BMW', 'codigo': '7'},
 {'nome': 'BRM', 'codigo': '8'},
 {'nome': 'Bugre', 'codigo': '123'},
 {'nome': 'Cadillac', 'codigo': '10'},
 {'nome': 'CBT Jipe', 'codigo': '11'},
 {'nome': 'CHANA', 'codigo': '136'},
 {'nome': 'CHANGAN', 'codigo': '182'},
 {'nome': 'CHERY', 'codigo': '161'},
 {'nome': 'Chrysler', 'codigo': '12'},
 {'nome': 'Citroën', 'codigo': '13'},
 {'nome': 'Cross Lander', 'codigo': '14'},
 {'nome': 'Daewoo', 'codigo': '15'},
 {'nome': 'Daihatsu', 'codigo': '16'},
 {'nome': 'Dodge', 'codigo': '17'},
 {'nome': 'EFFA', 'codigo': '147'},
 {'nome': 'Engesa', 'codigo': '18'},
 {'nome': 'Envemo', 'codigo': '19'},
 {'nome': 'Ferrari', 'codigo': '20'},
 {'nome': 'Fiat', 'codig

Esta api tiene una columna en donde a cada marca de coche le añade un código, de tal manera que puedes indentificar cada marca por su código.

In [32]:
datacars = pd.DataFrame(results)
datacars.head()

,codigo,nome
0,1,Acura
1,2,Agrale
2,3,Alfa Romeo
3,4,AM Gen
4,5,Asia Motors


Cambio el nombre de la columna 'nome' por el de 'make', ya que luego para hacer el merge de ambos dataset me va a ser necesario que se llamen iguales ambas columnas.

In [33]:
datacars.rename(columns={'nome': 'make'}, inplace=True)
datacars['make'] = datacars['make'].str.replace('Alfa Romeo', 'Alfa')
datacars.head()

,codigo,make
0,1,Acura
1,2,Agrale
2,3,Alfa
3,4,AM Gen
4,5,Asia Motors


Uno ambos datasets, para ellos utilizo un .merge y las unifico por la columna 'make'. De esta forma consigo que a cada marca de coche que tenía en el primer dataset se le añada su código correspondiente proveniente del segundo dataset.

In [34]:
s1 = pd.merge(cars1,datacars,how='left', on=['make'])
s1.head()

,make,model,power,gear_type,fuel_type,kms,old_years,price,codigo
0,Porsche,911,210.0,manual,gasoline,202000.0,20.0,999999,47
1,Ford,Mustang,487.0,manual,gasoline,30000.0,4.0,685000,22
2,Porsche,911,220.0,manual,gasoline,56300.0,30.0,555555,47
3,Porsche,911,368.0,manual,gasoline,2800.0,1.0,470000,47
4,Porsche,911,515.0,NaN,gasoline,10.0,0.0,450000,47


Una vez creada esta tabla, hago una pivot_table a través de 'make', 'codigo' y 'model', en donde a cada 'model' le indico los kilómetros de media, la antigüedad en años de media, la potencia media y el precio medio que tienen todos los modelos de coche de cada marca y código.

In [35]:
table = pd.pivot_table(s1, index=['make','codigo','model'],
                       aggfunc={'model':[len],'power': np.mean, 'kms':np.mean, 'old_years':np.mean,
                                'price':np.mean}).round()
table.head()

kms model old_years  power    price
                                mean   len      mean   mean     mean
make codigo model                                                   
Alfa 3      RomeoGiulietta   55684.0   291       3.0   92.0  13943.0
            RomeoMito        67039.0   132       6.0   76.0   9185.0
Audi 6      A1               37728.0   968       3.0   75.0  15536.0
            A3               87223.0  2091       5.0   92.0  16259.0
            A4              102455.0  2115       6.0  113.0  18778.0

Reordeno las columnas de una manera que me gusta más y que creo que tiene mayor sentido a la hora de leerlo.

In [36]:
table1 = table[['model','kms','power','old_years','price']]
table1.head()

model       kms  power old_years    price
                             len      mean   mean      mean     mean
make codigo model                                                   
Alfa 3      RomeoGiulietta   291   55684.0   92.0       3.0  13943.0
            RomeoMito        132   67039.0   76.0       6.0   9185.0
Audi 6      A1               968   37728.0   75.0       3.0  15536.0
            A3              2091   87223.0   92.0       5.0  16259.0
            A4              2115  102455.0  113.0       6.0  18778.0

Esta tabla es el resultado final de mi análisis. Viéndola entera, podemos sacar un resumen claro y detallado de las marcas y modelos de coches que están en el mercado de segunda mano en España. Indico el número de coches que hay de cada modelo, donde podemos ver cuales abundan más o cuales son más exclusivos, y de cada modelo podemos ver las medias de los datos de los kilómetros, la potencia, la antigüedad y el precio de cada modelo.

En el archivo Pipeline_Rafael_Eguren.py que entrego junto con este jupyter notebook, he creado un pipeline con todas las funciones utilizadas y de donde podemos extraer este mismo resultado.